<a href="https://colab.research.google.com/github/omar4646/My_repository/blob/main/princ_Econ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
!pip install pyreadr
import statsmodels.formula.api as smf
# Colab cell (bash)
!mkdir -p /content/rda
# Colab cell (bash)
!git clone https://github.com/ccolonescu/POE5Rdata.git /content/POE5Rdata

# copy all R data files (case-insensitive extensions)
!find /content/POE5Rdata/data -type f \( -iname "*.rda" -o -iname "*.rdata" -o -iname "*.RData" \) -exec cp -v {} /content/rda/ \;

# sanity check
!ls -lh /content/rda | head -n 50


fatal: destination path '/content/POE5Rdata' already exists and is not an empty directory.
'/content/POE5Rdata/data/coke.rda' -> '/content/rda/coke.rda'
'/content/POE5Rdata/data/nls.rda' -> '/content/rda/nls.rda'
'/content/POE5Rdata/data/vegas5.rda' -> '/content/rda/vegas5.rda'
'/content/POE5Rdata/data/bachelor.rda' -> '/content/rda/bachelor.rda'
'/content/POE5Rdata/data/okun5_aus.rda' -> '/content/rda/okun5_aus.rda'
'/content/POE5Rdata/data/bweight_small.rda' -> '/content/rda/bweight_small.rda'
'/content/POE5Rdata/data/njmin3.rda' -> '/content/rda/njmin3.rda'
'/content/POE5Rdata/data/pricoll.rda' -> '/content/rda/pricoll.rda'
'/content/POE5Rdata/data/inequality.rda' -> '/content/rda/inequality.rda'
'/content/POE5Rdata/data/qtm.rda' -> '/content/rda/qtm.rda'
'/content/POE5Rdata/data/mc20.rda' -> '/content/rda/mc20.rda'
'/content/POE5Rdata/data/mc1_fixed_x.rda' -> '/content/rda/mc1_fixed_x.rda'
'/content/POE5Rdata/data/apap.rda' -> '/content/rda/apap.rda'
'/content/POE5Rdata/data/insure

In [16]:
import pyreadr

chemical = pyreadr.read_r('/content/POE5Rdata/data/chemical2.rda')

In [17]:
chemical = chemical['chemical2']

In [18]:
chemical

,year,firm,lsales,lcapital,llabor,sk_labor,lmaterials,foreign,export,intangibles,ownership
0,2004,1,10.421390,9.848661,5.308268,0.267327,9.964441,0.25,1,1,0
1,2005,1,10.879330,10.065050,5.438079,0.267327,10.453630,0.25,1,1,0
2,2006,1,11.282740,10.093900,5.493062,0.267327,10.836850,0.25,1,1,0
3,2004,2,9.355739,8.774158,4.025352,0.000000,8.940368,0.00,0,0,0
4,2005,2,9.313799,8.874867,4.025352,0.000000,9.021598,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
595,2005,199,9.758462,8.872066,4.700480,0.016667,9.233275,0.00,0,0,0
596,2006,199,9.895959,9.052516,4.787492,0.016667,9.248406,0.00,0,0,0
597,2004,200,8.836374,7.649693,4.007333,0.000000,8.326758,0.00,0,1,0
598,2005,200,9.126959,8.143517,4.007333,0.000000,8.830981,0.00,0,0,0


In [19]:
import statsmodels.formula.api as smf

chemical["d_lnSALES"] = chemical.groupby('firm')["lsales"].diff(periods=1)
chemical["d_lnCAPITAL"] = chemical.groupby('firm')["lcapital"].diff(periods=1)
chemical["d_lnLABOR"] = chemical.groupby('firm')["llabor"].diff(periods=1)

formula = 'd_lnSALES ~ d_lnCAPITAL + d_lnLABOR'
chemical_diff = chemical.dropna(subset=["d_lnSALES", "d_lnCAPITAL", "d_lnLABOR"]).copy()
res_fd = smf.ols(formula, data=chemical_diff).fit(
    cov_type="cluster", cov_kwds={"groups": chemical_diff['firm']}
)
print("\n------ First-difference OLS results (clustered SE by id) ------")
print(res_fd.summary())


------ First-difference OLS results (clustered SE by id) ------
                            OLS Regression Results                            
Dep. Variable:              d_lnSALES   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     16.06
Date:                Mon, 15 Sep 2025   Prob (F-statistic):           3.42e-07
Time:                        15:04:17   Log-Likelihood:                -160.65
No. Observations:                 400   AIC:                             327.3
Df Residuals:                     397   BIC:                             339.3
Df Model:                           2                                         
Covariance Type:              cluster                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------

In [20]:
vars_to_demean = ["lsales", "lcapital", "llabor"]
for v in vars_to_demean:
    mean_name = f"{v}_mean"
    difference = f"differ_{v}"
    chemical[mean_name] = chemical.groupby('firm')[v].transform("mean")
    chemical[difference] = chemical[v] - chemical[mean_name]
# ---- Within (demeaned) OLS: tilde_y ~ tilde_cap + tilde_lab  (no intercept) ----
formula = "differ_lsales ~ differ_lcapital + differ_llabor - 1"
print("\nRunning within OLS (formula):", formula)
res_within = smf.ols(formula, data=chemical).fit(cov_type="cluster", cov_kwds={"groups": chemical['firm']})
print(res_within.summary())


Running within OLS (formula): differ_lsales ~ differ_lcapital + differ_llabor - 1
                                 OLS Regression Results                                
Dep. Variable:          differ_lsales   R-squared (uncentered):                   0.125
Model:                            OLS   Adj. R-squared (uncentered):              0.122
Method:                 Least Squares   F-statistic:                              19.49
Date:                Mon, 15 Sep 2025   Prob (F-statistic):                    1.86e-08
Time:                        15:04:17   Log-Likelihood:                          5.8503
No. Observations:                 600   AIC:                                     -7.701
Df Residuals:                     598   BIC:                                      1.093
Df Model:                           2                                                  
Covariance Type:              cluster                                                  
                      coef    std err

Pooled OLS Model (Restricted Model)

The pooled OLS model assumes no fixed effects (i.e., ignores the unobserved heterogeneity).

In [21]:
import statsmodels.formula.api as smf
df = chemical.set_index(['firm', 'year'])
df = df.sort_index()
# Estimate the Pooled OLS model (no fixed effects)
pooled_model = smf.ols('lsales ~ lcapital + llabor', data=df).fit()

# Pooled OLS results
print(pooled_model.summary())


                            OLS Regression Results                            
Dep. Variable:                 lsales   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.498
Method:                 Least Squares   F-statistic:                     298.0
Date:                Mon, 15 Sep 2025   Prob (F-statistic):           1.79e-90
Time:                        15:04:17   Log-Likelihood:                -748.36
No. Observations:                 600   AIC:                             1503.
Df Residuals:                     597   BIC:                             1516.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.8797      0.171     34.368      0.0

Fixed Effects Model (Unrestricted Model)

Now, estimate the fixed effects model, which accounts for unobserved heterogeneity using within transformation.

In [22]:
!pip install linearmodels
from linearmodels.panel import PanelOLS

# Fixed Effects model
fe_model = PanelOLS.from_formula('lsales ~ lcapital + llabor + EntityEffects', data=df)
fe_res = fe_model.fit()

# Fixed Effects results
print(fe_res)


                          PanelOLS Estimation Summary                           
Dep. Variable:                 lsales   R-squared:                        0.1252
Estimator:                   PanelOLS   R-squared (Between):              0.4106
No. Observations:                 600   R-squared (Within):               0.1252
Date:                Mon, Sep 15 2025   R-squared (Overall):              0.4104
Time:                        15:04:27   Log-likelihood                    5.8503
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      28.491
Entities:                         200   P-value                           0.0000
Avg Obs:                       3.0000   Distribution:                   F(2,398)
Min Obs:                       3.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             28.491
                            

Compute SSE for Both Models

Next, you will need to compute the sum of squared residuals (SSE) for both the fixed effects and pooled OLS models.

In [23]:
# Sum of squared residuals for Pooled OLS model
SSE_R = sum((pooled_model.fittedvalues - df['lsales'])**2)

# Sum of squared residuals for Fixed Effects model (using the correct method to access fitted values)
# Align indices before computing the difference
fe_predicted = fe_res.predict()
SSE_U = sum((fe_predicted['fitted_values'] - df['lsales'])**2)


print(f"SSE_R (Pooled OLS): {SSE_R}")
print(f"SSE_U (Fixed Effects): {SSE_U}")

SSE_R (Pooled OLS): 425.63655733503504
SSE_U (Fixed Effects): 34956.62973313458


In [24]:
chemical_three = pyreadr.read_r('/content/POE5Rdata/data/chemical3.rda')
chemical3 = chemical_three['chemical3']

In [25]:
chemical3

,year,firm,lsales,lcapital,llabor,sk_labor,lmaterials,foreign,export,intangibles,ownership
0,2004,1,10.421390,9.848661,5.308268,0.267327,9.964441,0.25,1,1,0
1,2005,1,10.879330,10.065050,5.438079,0.267327,10.453630,0.25,1,1,0
2,2006,1,11.282740,10.093900,5.493062,0.267327,10.836850,0.25,1,1,0
3,2004,2,9.355739,8.774158,4.025352,0.000000,8.940368,0.00,0,0,0
4,2005,2,9.313799,8.874867,4.025352,0.000000,9.021598,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2995,2005,999,11.646140,8.720950,4.997212,0.387755,11.167050,0.00,0,0,0
2996,2006,999,11.849400,9.740969,5.010635,0.387755,11.430090,0.00,0,0,0
2997,2004,1000,11.898800,11.339080,5.971262,0.145408,11.663720,0.00,1,1,0
2998,2005,1000,12.062720,11.497810,5.945421,0.145408,11.823320,0.00,1,1,0


In [26]:
# 2) Specify formula (pooled OLS) ----------------------------------------
formula = 'lsales ~ lcapital + llabor'   # intercept included by default

# 3) Pooled OLS - conventional (non-robust) -------------------------------
pooled = smf.ols(formula, data=chemical3).fit()   # conventional OLS
print("\nConventional OLS summary:")
print(pooled.summary())

# 4) Pooled OLS - heteroskedasticity-robust (HC1) -------------------------
pooled_hc = pooled.get_robustcov_results(cov_type='HC1')   # or 'HC3'
print("\nHeteroskedasticity-robust (HC1) summary:")
print(pooled_hc.summary())

# 5) Pooled OLS - cluster-robust by firm ---------------------------------
pooled_cl = pooled.get_robustcov_results(cov_type='cluster',
                                         groups=chemical3['firm'])
print("\nCluster-robust (cluster by firm) summary:")
print(pooled_cl.summary())


Conventional OLS summary:
                            OLS Regression Results                            
Dep. Variable:                 lsales   R-squared:                       0.558
Model:                            OLS   Adj. R-squared:                  0.558
Method:                 Least Squares   F-statistic:                     1892.
Date:                Mon, 15 Sep 2025   Prob (F-statistic):               0.00
Time:                        15:04:27   Log-Likelihood:                -3836.8
No. Observations:                3000   AIC:                             7680.
Df Residuals:                    2997   BIC:                             7698.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.5408    

Using Fixed Effects and Cluster-Robust Standard Errors
for a Production Functio

In [27]:
chemical3.columns

Index(['year', 'firm', 'lsales', 'lcapital', 'llabor', 'sk_labor',
       'lmaterials', 'foreign', 'export', 'intangibles', 'ownership'],
      dtype='object')

In [28]:
y = chemical3["lsales"]
X = chemical3[["lcapital", "llabor"]]

# Set a MultiIndex on chemical3
chemical3 = chemical3.set_index(['firm', 'year'])
chemical3 = chemical3.sort_index()

y = chemical3["lsales"]
X = chemical3[["lcapital", "llabor"]]

# === 6) FE with conventional (unadjusted) SEs ===
res_conv = PanelOLS(y, X, entity_effects=True).fit(cov_type="unadjusted")

# === 7) FE with cluster-robust SEs (cluster by firm/entity) ===
res_clu  = PanelOLS(y, X, entity_effects=True).fit(cov_type="clustered", cluster_entity=True)

print("\n=== FIXED EFFECTS (conventional SEs) ===")
print(res_conv.summary)

print("\n=== FIXED EFFECTS (cluster-robust by firm) ===")
print(res_clu.summary)


=== FIXED EFFECTS (conventional SEs) ===
                          PanelOLS Estimation Summary                           
Dep. Variable:                 lsales   R-squared:                        0.0582
Estimator:                   PanelOLS   R-squared (Between):              0.3977
No. Observations:                3000   R-squared (Within):               0.0582
Date:                Mon, Sep 15 2025   R-squared (Overall):              0.3974
Time:                        15:04:27   Log-likelihood                   -681.88
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      61.724
Entities:                        1000   P-value                           0.0000
Avg Obs:                       3.0000   Distribution:                  F(2,1998)
Min Obs:                       3.0000                                           
Max Obs:                       3.0000   F-statistic (robust):      

In [29]:
chemical3

lsales   lcapital    llabor  sk_labor  lmaterials  foreign  \
firm year                                                                  
1    2004  10.421390   9.848661  5.308268  0.267327    9.964441     0.25   
     2005  10.879330  10.065050  5.438079  0.267327   10.453630     0.25   
     2006  11.282740  10.093900  5.493062  0.267327   10.836850     0.25   
2    2004   9.355739   8.774158  4.025352  0.000000    8.940368     0.00   
     2005   9.313799   8.874867  4.025352  0.000000    9.021598     0.00   
...              ...        ...       ...       ...         ...      ...   
999  2005  11.646140   8.720950  4.997212  0.387755   11.167050     0.00   
     2006  11.849400   9.740969  5.010635  0.387755   11.430090     0.00   
1000 2004  11.898800  11.339080  5.971262  0.145408   11.663720     0.00   
     2005  12.062720  11.497810  5.945421  0.145408   11.823320     0.00   
     2006  11.289530  11.907880  6.001415  0.145408   10.995230     0.00   

           export  intangibles  ownership  
firm year                                  
1    2004       1            1          0  
     2005       1            1          0  
     2006       1            1          0  
2    2004       0            0          0  
     2005       0            0          0  
...           ...          ...        ...  
999  2005       0            0          0  
     2006       0            0          0  
1000 2004       1            1          0  
     2005       1            1          0  
     2006       1            1          0  

[3000 rows x 9 columns]

In [30]:
# Your existing objects:
# y = chemical3["lsales"]
# X = chemical3[["lcapital", "llabor"]]

from linearmodels.panel import PanelOLS

res_conv = PanelOLS(y, X, entity_effects=True).fit(cov_type="unadjusted")
res_clu  = PanelOLS(y, X, entity_effects=True).fit(cov_type="clustered", cluster_entity=True)

# ----- Recover an overall intercept (grand-mean normalization) -----
import numpy as np

beta = res_clu.params                     # coefficients on lcapital, llabor
const_hat = y.mean() - np.dot(X.mean(), beta)

print("Recovered constant (overall intercept):", float(const_hat))

# If you want to add it to a tiny table:
import pandas as pd
tbl = pd.DataFrame({
    "coef": res_clu.params,
    "std_err": res_clu.std_errors,
    "t": res_clu.tstats
})
tbl.loc["const(recovered)","coef"] = const_hat
print("\nCluster-robust FE results + recovered constant:\n", tbl)


Recovered constant (overall intercept): 7.946273986792225

Cluster-robust FE results + recovered constant:
                       coef   std_err         t
lcapital          0.116040  0.027329  4.246089
llabor            0.268880  0.045786  5.872494
const(recovered)  7.946274       NaN       NaN


Random Effects Estimation of a Production Function

In [31]:
chemical3.reset_index(inplace=True)

In [32]:
# === Random Effects (FGLS) for chemical3 with FGLS SEs and Cluster-Robust SEs ===
import numpy as np, pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS # Import PanelOLS for Fixed Effects

# Assumes you already have a DataFrame `chemical3` with columns:
# 'firm','year','lsales','lcapital','llabor'
# If logs not present, create them and rename accordingly.

df = chemical3.copy()

# Ensure balanced-ish panel metadata
ID, TIME = "firm", "year"
YCOL, XCOLS = "lsales", ["lcapital", "llabor"]

# Sort and set (firm,year) index
df = df[[ID, TIME, YCOL] + XCOLS].dropna().sort_values([ID, TIME]).set_index([ID, TIME])

# Count periods per firm (T); Example 15.9 uses T=3; code handles unbalanced too
Ti = df.groupby(level=0).size()
# We'll use each firm's own T for moments; if perfectly balanced, Ti.nunique()==1
# Add a constant (intercept) for pooled OLS and the transformed model
def add_const(M):
    return sm.add_constant(M, has_constant="add")

# ---------- 1) Pooled OLS residuals (for Swamy–Arora variance components) ----------
y = df[YCOL]
X = add_const(df[XCOLS])
ols = sm.OLS(y, X).fit()

ehat = ols.resid
# Firm means of residuals and counts
eibar = ehat.groupby(level=0).mean()
Ti = ehat.groupby(level=0).size()

# Within (idiosyncratic) variance: sum over (e_it - ebar_i)^2 / (sum_i (T_i-1) - Kx)
demeaned = ehat - ehat.groupby(level=0).transform("mean")
Kx = X.shape[1]                          # includes constant
den_within = (Ti - 1).sum() - (Kx - 1)   # subtract slopes, constant handled by between; mild dof choice
sigma2_e = (demeaned**2).sum() / den_within

# Between variance of firm means: max(0, Var(ē_i) - sigma2_e / T_i averaged)
# Use weighted average of 1/T_i for unbalanced panels
w = 1.0 / Ti
sigma2_u = ( (eibar**2).mean() - (sigma2_e * w.mean()) ).clip(min=0)

# ---------- 2) Compute alpha for partial demeaning ----------
# If panel is unbalanced, replace T by firm-specific T_i inside the transform.
# For a simple, Wooldridge-style single-alpha, use average T:
Tbar = Ti.mean()
alpha = 1.0 - np.sqrt(sigma2_e) / np.sqrt(Tbar * sigma2_u + sigma2_e)

# Guardrails
alpha = float(np.clip(alpha, 0.0, 1.0))

print(f"Estimated components: sigma_u^2={sigma2_u:.6f}, sigma_e^2={sigma2_e:.6f}")
print(f"Partial-demeaning coefficient alpha = {alpha:.4f}")

# ---------- 3) Build RE-transformed (quasi-demeaned) variables ----------
# y* = y - alpha * ybar_i,   x* = x - alpha * xbar_i,   const* = 1 - alpha
group = df.groupby(level=0)
y_star = y - alpha * group[YCOL].transform("mean")

X_star = X.copy()
for col in X_star.columns:
    if col == "const":                  # mean of constant is 1, so const* = 1 - alpha
        X_star[col] = 1.0 - alpha
    else:
        X_star[col] = X_star[col] - alpha * group[col].transform("mean")

# ---------- 4) OLS on transformed data (== RE / FGLS) ----------
# (a) Model-based FGLS SEs (non-robust)
re_fgls = sm.OLS(y_star, X_star).fit()

# (b) Cluster-robust SEs by firm (conservative)
firm_index = X_star.index.get_level_values(0)
re_clus = sm.OLS(y_star, X_star).fit(cov_type="cluster", cov_kwds={"groups": firm_index})

# ---------- 5) Fixed Effects model using PanelOLS ----------
# Use the original y and X without the added constant for PanelOLS with EntityEffects
# Add a constant term explicitly for PanelOLS
X_fe = sm.add_constant(df[XCOLS], has_constant='add')
fe_model = PanelOLS(df[YCOL], X_fe, entity_effects=True).fit(cov_type="clustered", cluster_entity=True)


# ---------- 6) Print Summaries and Compact comparison table ----------
print("\n=== RANDOM EFFECTS (Cluster-Robust SEs) ===")
print(re_clus.summary())

print("\n=== FIXED EFFECTS (Cluster-Robust SEs) ===")
print(fe_model.summary)


def tidy(res, tag):
    out = pd.DataFrame({"coef": res.params, "se": res.bse})
    out["t"] = out["coef"] / out["se"]
    out["spec"] = tag
    return out

# For the FE model from linearmodels, the summary object contains the parameters, std errors and t-stats
fe_params = fe_model.params
fe_std_errors = fe_model.std_errors
fe_tstats = fe_model.tstats

fe_tidy = pd.DataFrame({
    "coef": fe_params,
    "se": fe_std_errors,
    "t": fe_tstats
})
fe_tidy["spec"] = "fe_clus"


tbl = pd.concat([tidy(re_clus, "re_clus"), fe_tidy]).reset_index().rename(columns={"index":"param"})
print("\nRandom Effects vs Fixed Effects — coefficients and t-stats (Cluster SEs):\n")
print(tbl.pivot(index="param", columns="spec", values=["coef","se","t"]).round(4))

Estimated components: sigma_u^2=0.607958, sigma_e^2=0.147907
Partial-demeaning coefficient alpha = 0.7261

=== RANDOM EFFECTS (Cluster-Robust SEs) ===
                            OLS Regression Results                            
Dep. Variable:                 lsales   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     470.4
Date:                Mon, 15 Sep 2025   Prob (F-statistic):          1.11e-144
Time:                        15:04:28   Log-Likelihood:                -1371.5
No. Observations:                3000   AIC:                             2749.
Df Residuals:                    2997   BIC:                             2767.
Df Model:                           2                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|     

In [33]:
chemical3.columns

Index(['firm', 'year', 'lsales', 'lcapital', 'llabor', 'sk_labor',
       'lmaterials', 'foreign', 'export', 'intangibles', 'ownership'],
      dtype='object')

In [34]:
df.columns

Index(['lsales', 'lcapital', 'llabor'], dtype='object')

# $$\text{Calculating } \hat{\sigma}_e^2 \text{ (`Within` Variance)}$$
Idiosyncratic shock variance

$$
\hat{\sigma_e}^2 = \frac{\sum_{i=1}^{n} \sum_{t=1}^{T_i} \left( e_{it} - \bar{e}_i \right)^2}{\sum_{i=1}^{n} \left( T_i - 1 \right) - K_x}
$$


# Swamy–Arora between variance estimator
Unobserved heterogeneity variance

$$
\hat{\sigma}_u^2 = \max \left\{ 0, \frac{1}{n} \sum_{i=1}^{n} \bar{e}_i^2 - \hat{\sigma}_e^2 \cdot \frac{1}{n} \sum_{i=1}^{n} \frac{1}{T_i} \right\}
$$


RE quasi-demeaning weight. It tells you how much of the unit mean to subtract — 0 means none (pooled), 1 means full (FE), and values in between produce the optimal (GLS/RE) compromise.

$$
\hat{\sigma}_e^2 = \frac{\sum_{i=1}^{n} \sum_{t=1}^{T_i} (e_{it} - \bar{e}_i)^2}{\sum_{i=1}^{n} (T_i - 1) - K_x}
$$


# Random Effects Estimation of a Wage Equation


In [35]:
nls_raw = pyreadr.read_r('/content/POE5Rdata/data/nls_panel.rda')
nls = nls_raw['nls_panel']

In [36]:
nls.columns

Index(['id', 'year', 'lwage', 'hours', 'age', 'educ', 'collgrad', 'msp',
       'nev_mar', 'not_smsa', 'c_city', 'south', 'black', 'union', 'exper',
       'exper2', 'tenure', 'tenure2'],
      dtype='object')

# Fixed and Random Effects Estimates of a Wage Equation

In [37]:
# === Random Effects (FGLS) for nls with FGLS SEs and Cluster-Robust SEs ===
import numpy as np, pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS # Import PanelOLS for Fixed Effects

# Assumes you already have a DataFrame `nls` with columns:
# 'firm', 'year', 'lwage', 'educ', 'exper', 'tenure', 'south', 'black', 'union'
df_nls = nls.copy()

# Create squared terms for experience and tenure
df_nls['exper_squared'] = df_nls['exper'] ** 2
df_nls['tenure_squared'] = df_nls['tenure'] ** 2

# Ensure balanced-ish panel metadata
ID, TIME = "id", "year"
YCOL, XCOLS = "lwage", ["educ", 'exper', 'exper_squared', 'tenure', 'tenure_squared', 'south', 'black', 'union']

# Sort and set (firm,year) index
df_nls = df_nls[[ID, TIME, YCOL] + XCOLS].dropna().sort_values([ID, TIME]).set_index([ID, TIME])

# Count periods per firm (T); Example 15.9 uses T=3; code handles unbalanced too
Ti = df_nls.groupby(level=0).size()
# We'll use each firm's own T for moments; if perfectly balanced, Ti.nunique()==1
# Add a constant (intercept) for pooled OLS and the transformed model
def add_const(M):
    return sm.add_constant(M, has_constant="add")

# ---------- 1) Pooled OLS residuals (for Swamy–Arora variance components) ----------
y = df_nls[YCOL]
X = add_const(df_nls[XCOLS])
ols = sm.OLS(y, X).fit()

ehat = ols.resid
# Firm means of residuals and counts
eibar = ehat.groupby(level=0).mean()
Ti = ehat.groupby(level=0).size()

# Within (idiosyncratic) variance: sum over (e_it - ebar_i)^2 / (sum_i (T_i-1) - Kx)
demeaned = ehat - ehat.groupby(level=0).transform("mean")
Kx = X.shape[1]                          # includes constant
den_within = (Ti - 1).sum() - (Kx - 1)   # subtract slopes, constant handled by between; mild dof choice
sigma2_e = (demeaned**2).sum() / den_within

# Between variance of firm means: max(0, Var(ē_i) - sigma2_e / T_i averaged)
# Use weighted average of 1/T_i for unbalanced panels
w = 1.0 / Ti
sigma2_u = ( (eibar**2).mean() - (sigma2_e * w.mean()) ).clip(min=0)

# ---------- 2) Compute alpha for partial demeaning ----------
# If panel is unbalanced, replace T by firm-specific T_i inside the transform.
# For a simple, Wooldridge-style single-alpha, use average T:
Tbar = Ti.mean()
alpha = 1.0 - np.sqrt(sigma2_e) / np.sqrt(Tbar * sigma2_u + sigma2_e)

# Guardrails
alpha = float(np.clip(alpha, 0.0, 1.0))

print(f"Estimated components: sigma_u^2={sigma2_u:.6f}, sigma_e^2={sigma2_e:.6f}")
print(f"Partial-demeaning coefficient alpha = {alpha:.4f}")

# ---------- 3) Build RE-transformed (quasi-demeaned) variables ----------
# y* = y - alpha * ybar_i,   x* = x - alpha * xbar_i,   const* = 1 - alpha
group = df_nls.groupby(level=0)
y_star = y - alpha * group[YCOL].transform("mean")

X_star = X.copy()
for col in X_star.columns:
    if col == "const":                  # mean of constant is 1, so const* = 1 - alpha
        X_star[col] = 1.0 - alpha
    else:
        X_star[col] = X_star[col] - alpha * group[col].transform("mean")

# ---------- 4) OLS on transformed data (== RE / FGLS) ----------
# (a) Model-based FGLS SEs (non-robust)
re_fgls = sm.OLS(y_star, X_star).fit()

# ---------- 5) Fixed Effects model using PanelOLS ----------
# Use the original y and X without the added constant for PanelOLS with EntityEffects
# Add a constant term explicitly for PanelOLS
X_fe = sm.add_constant(df_nls[XCOLS], has_constant='add')
fe_model = PanelOLS(df_nls[YCOL], X_fe, entity_effects=True, drop_absorbed=True).fit()

# ---------- 6) Print Summaries and Compact comparison table ----------
print("\n=== RANDOM EFFECTS (Cluster-Robust SEs) ===")
print(re_fgls.summary())

print("\n=== FIXED EFFECTS (Cluster-Robust SEs) ===")
print(fe_model.summary)

def tidy(res, tag):
    out = pd.DataFrame({"coef": res.params, "se": res.bse})
    out["t"] = out["coef"] / out["se"]
    out["spec"] = tag
    return out

# For the FE model, summarize the results
fe_params = fe_model.params
fe_std_errors = fe_model.std_errors
fe_tstats = fe_model.tstats

fe_tidy = pd.DataFrame({
    "coef": fe_params,
    "se": fe_std_errors,
    "t": fe_tstats
})
fe_tidy["spec"] = "fe_clus"

# Combine and compare Random Effects vs Fixed Effects
tbl = pd.concat([tidy(re_fgls, "re_fgls"), fe_tidy]).reset_index().rename(columns={"index":"param"})
print("\nRandom Effects vs Fixed Effects — coefficients and t-stats (Cluster SEs):\n")
print(tbl.pivot(index="param", columns="spec", values=["coef","se","t"]).round(4))


Estimated components: sigma_u^2=0.106828, sigma_e^2=0.038797
Partial-demeaning coefficient alpha = 0.7398

=== RANDOM EFFECTS (Cluster-Robust SEs) ===
                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.195
Model:                            OLS   Adj. R-squared:                  0.194
Method:                 Least Squares   F-statistic:                     108.3
Date:                Mon, 15 Sep 2025   Prob (F-statistic):          2.22e-162
Time:                        15:04:28   Log-Likelihood:                 756.88
No. Observations:                3580   AIC:                            -1496.
Df Residuals:                    3571   BIC:                            -1440.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t| 

/tmp/ipython-input-280793172.py:81: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

educ, black

  fe_model = PanelOLS(df_nls[YCOL], X_fe, entity_effects=True, drop_absorbed=True).fit()


In [38]:
import numpy as np, pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

# Assumes you have `chemical3` dataset with columns: 'firm', 'year', 'lsales', 'lcapital', 'llabor'

df_chem3 = chemical3.copy()

# Set panel metadata (firm, year), dependent variable (lsales), and independent variables (lcapital, llabor)
ID, TIME = "firm", "year"
YCOL, XCOLS = "lsales", ["lcapital", "llabor"]

# Sort the dataset and drop any rows with missing values, then set panel structure
df_chem3 = df_chem3[[ID, TIME, YCOL] + XCOLS].dropna().sort_values([ID, TIME]).set_index([ID, TIME])

# Add a constant (intercept) for pooled OLS and the transformed model
def add_const(M):
    return sm.add_constant(M, has_constant="add")

# Run Pooled OLS regression
y = df_chem3[YCOL]
X = add_const(df_chem3[XCOLS])
ols = sm.OLS(y, X).fit()

# Save the residuals from Pooled OLS regression
ehat = ols.resid

# Group by firm to compute firm-specific residual means and variance
eibar = ehat.groupby(level=0).mean()
Ti = ehat.groupby(level=0).size()

# Compute within-variance (idiosyncratic variance) and between-variance
demeaned = ehat - ehat.groupby(level=0).transform("mean")
Kx = X.shape[1]  # Number of independent variables (including constant)
den_within = (Ti - 1).sum() - (Kx - 1)  # Degrees of freedom for within variance
sigma2_e = (demeaned**2).sum() / den_within

# Compute between-variance (firm-level residual variance)
w = 1.0 / Ti
sigma2_u = ( (eibar**2).mean() - (sigma2_e * w.mean()) ).clip(min=0)

# Compute the LM test statistic
Tbar = Ti.mean()  # Average time periods per firm
# Compute the LM test statistic
LM = np.sqrt((df.shape[0] * Tbar) / (2 * (Tbar - 1))) * ((np.sum(ehat**2) / np.sum(eibar**2)) - 1)

# Print LM test statistic
print(f"LM Test Statistic = {LM:.4f}")

# Critical value for alpha = 0.05 (use chi-squared distribution with 1 degree of freedom)
critical_value = 3.841

# Decision Rule
if LM > critical_value:
    print("Reject H0: Random Effects are present.")
else:
    print("Fail to reject H0: No evidence of Random Effects.")


LM Test Statistic = 116.1958
Reject H0: Random Effects are present.


# Breusch-Pagan Lagrange Multiplier (LM) Test for Random Effects

The Breusch-Pagan LM test is used to choose between pooled OLS and Random Effects. The null hypothesis ($H_0$) is that the variance of the individual effects is zero, meaning there are no panel-specific effects (pooled OLS is appropriate). The alternative hypothesis ($H_1$) is that the variance of the individual effects is greater than zero (Random Effects is appropriate).

The formula for the LM test statistic is:

$$
LM = \sqrt{\frac{NT}{2(T-1)}} \left( \frac{\sum_{i=1}^{n} \sum_{t=1}^{T_i} \hat{e}_{it}^2}{\sum_{i=1}^{n} T_i \hat{\bar{e}}_i^2} - 1 \right)
$$

Where:
- $N$ is the number of individuals (firms).
- $T$ is the number of time periods (assuming a balanced panel for this version of the formula).
- $T_i$ is the number of time periods for individual $i$.
- $\hat{e}_{it}$ are the residuals from the pooled OLS regression.
- $\hat{\bar{e}}_i$ are the means of the pooled OLS residuals for each individual $i$.

Alternatively, a more common form when $\sigma_u^2$ and $\sigma_e^2$ are estimated is:

$$
LM = \frac{NT}{2(T-1)} \left[ \frac{\sum_{i=1}^N (\sum_{t=1}^T \hat{e}_{it})^2}{\sum_{i=1}^N \sum_{t=1}^T \hat{e}_{it}^2} - 1 \right]^2 \sim \chi^2(1)
$$

In [39]:
# === Random Effects (FGLS) for chemical3 with FGLS SEs and Cluster-Robust SEs ===
import numpy as np, pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS # Import PanelOLS for Fixed Effects

# Assumes you already have a DataFrame `chemical3` with columns:
# 'firm','year','lsales','lcapital','llabor'
# If logs not present, create them and rename accordingly.

df = chemical3.copy()

# Ensure balanced-ish panel metadata
ID, TIME = "firm", "year"
YCOL, XCOLS = "lsales", ["lcapital", "llabor"]

# Sort and set (firm,year) index
df = df[[ID, TIME, YCOL] + XCOLS].dropna().sort_values([ID, TIME]).set_index([ID, TIME])

# Count periods per firm (T); Example 15.9 uses T=3; code handles unbalanced too
Ti = df.groupby(level=0).size()
# We'll use each firm's own T for moments; if perfectly balanced, Ti.nunique()==1
# Add a constant (intercept) for pooled OLS and the transformed model
def add_const(M):
    return sm.add_constant(M, has_constant="add")

# ---------- 1) Pooled OLS residuals (for Swamy–Arora variance components) ----------
y = df[YCOL]
X = add_const(df[XCOLS])
ols = sm.OLS(y, X).fit()

ehat = ols.resid
# Firm means of residuals and counts
eibar = ehat.groupby(level=0).mean()
Ti = ehat.groupby(level=0).size()

# Within (idiosyncratic) variance: sum over (e_it - ebar_i)^2 / (sum_i (T_i-1) - Kx)
demeaned = ehat - ehat.groupby(level=0).transform("mean")
Kx = X.shape[1]                          # includes constant
den_within = (Ti - 1).sum() - (Kx - 1)   # subtract slopes, constant handled by between; mild dof choice
sigma2_e = (demeaned**2).sum() / den_within

# Between variance of firm means: max(0, Var(ē_i) - sigma2_e / T_i averaged)
# Use weighted average of 1/T_i for unbalanced panels
w = 1.0 / Ti
sigma2_u = ( (eibar**2).mean() - (sigma2_e * w.mean()) ).clip(min=0)

# ---------- 2) Compute alpha for partial demeaning ----------
# If panel is unbalanced, replace T by firm-specific T_i inside the transform.
# For a simple, Wooldridge-style single-alpha, use average T:
Tbar = Ti.mean()
alpha = 1.0 - np.sqrt(sigma2_e) / np.sqrt(Tbar * sigma2_u + sigma2_e)

# Guardrails
alpha = float(np.clip(alpha, 0.0, 1.0))

print(f"Estimated components: sigma_u^2={sigma2_u:.6f}, sigma_e^2={sigma2_e:.6f}")
print(f"Partial-demeaning coefficient alpha = {alpha:.4f}")

# ---------- 3) Build RE-transformed (quasi-demeaned) variables ----------
# y* = y - alpha * ybar_i,   x* = x - alpha * xbar_i,   const* = 1 - alpha
group = df.groupby(level=0)
y_star = y - alpha * group[YCOL].transform("mean")

X_star = X.copy()
for col in X_star.columns:
    if col == "const":                  # mean of constant is 1, so const* = 1 - alpha
        X_star[col] = 1.0 - alpha
    else:
        X_star[col] = X_star[col] - alpha * group[col].transform("mean")

# ---------- 4) OLS on transformed data (== RE / FGLS) ----------
# (a) Model-based FGLS SEs (non-robust)
re_fgls = sm.OLS(y_star, X_star).fit() # Removed cov_type and cov_kwds

# (b) conventional SEs by firm (unadjusted)
firm_index = X_star.index.get_level_values(0)
re_unadj = sm.OLS(y_star, X_star).fit() # Removed cov_type and cov_kwds

# ---------- 5) Fixed Effects model using PanelOLS ----------
# Use the original y and X without the added constant for PanelOLS with EntityEffects
# Add a constant term explicitly for PanelOLS
X_fe = sm.add_constant(df[XCOLS], has_constant='add')
fe_model = PanelOLS(df[YCOL], X_fe, entity_effects=True).fit(cov_type="unadjusted")


# ---------- 6) Print Summaries and Compact comparison table ----------
print("\n=== RANDOM EFFECTS (Unadjusted SEs) ===")
print(re_unadj.summary())

print("\n=== FIXED EFFECTS (Unadjusted SEs) ===")
print(fe_model.summary)


def tidy(res, tag):
    out = pd.DataFrame({"coef": res.params, "se": res.bse})
    out["t"] = out["coef"] / out["se"]
    out["spec"] = tag
    return out

# For the FE model from linearmodels, the summary object contains the parameters, std errors and t-stats
fe_params = fe_model.params
fe_std_errors = fe_model.std_errors
fe_tstats = fe_model.tstats

fe_tidy = pd.DataFrame({
    "coef": fe_params,
    "se": fe_std_errors,
    "t": fe_tstats
})
fe_tidy["spec"] = "fe_unadj"


tbl = pd.concat([tidy(re_unadj, "re_unadj"), fe_tidy]).reset_index().rename(columns={"index":"param"})
print("\nRandom Effects vs Fixed Effects — coefficients and t-stats (Unadjusted SEs):\n")
print(tbl.pivot(index="param", columns="spec", values=["coef","se","t"]).round(4))
# =========================
# Hausman & Hausman-contrast tests
# =========================
from scipy.stats import chi2
import numpy as np
import pandas as pd

# 0) the slope names to compare (exclude the intercept)
slope_names = [c for c in XCOLS if (c in re_fgls.params.index) and (c in fe_model.params.index)]

# 1) Get a FE model with *conventional* (unadjusted) covariance to match the textbook derivation
# ❌ was: .fit(cov_type="unadjusted", drop_absorbed=True)
fe_unadj = PanelOLS(df[YCOL], X_fe, entity_effects=True) \
             .fit(cov_type="unadjusted")


# We’ll also collect the *clustered* covariances already fit:
#   - FE: fe_model (clustered by entity)
#   - RE: re_unadj (clustered by entity)

def hausman_joint(beta_fe, cov_fe, beta_re, cov_re, labels, tag):
    """Joint Hausman contrast: H = d' (Var_FE - Var_RE)^(-1) d"""
    # align as vector in the order of labels
    bfe = beta_fe.loc[labels].to_numpy()
    bre = beta_re.loc[labels].to_numpy()
    d   = bfe - bre

    Vfe = cov_fe.loc[labels, labels].to_numpy()
    Vre = cov_re.loc[labels, labels].to_numpy()
    V   = Vfe - Vre

    # Invert safely (positive-definiteness can fail w/robust covariances)
    try:
        Vinv = np.linalg.inv(V)
        inv_used = "standard inverse"
    except np.linalg.LinAlgError:
        Vinv = np.linalg.pinv(V)
        inv_used = "pseudo-inverse"

    H = float(d.T @ Vinv @ d)
    df = len(labels)
    p  = 1.0 - chi2.cdf(H, df)
    return {"H": H, "df": df, "p": p, "inv": inv_used, "d": d, "V": V}

def hausman_t_per_coef(beta_fe, se_fe, beta_re, se_re, labels):
    """Coefficient-by-coefficient t-tests: (b_FE - b_RE)/sqrt(se_FE^2 - se_RE^2)"""
    rows = []
    for k in labels:
        num = beta_fe[k] - beta_re[k]
        den2 = se_fe[k]**2 - se_re[k]**2
        tval = np.nan
        if den2 > 0:
            tval = num / np.sqrt(den2)
        rows.append({"var": k, "diff(b_FE-b_RE)": num, "t (unadj)": tval,
                     "se_FE": se_fe[k], "se_RE": se_re[k]})
    return pd.DataFrame(rows).set_index("var")

# 2) CONVENTIONAL (matches book)
# FE (unadjusted)
bfe_un  = fe_unadj.params
sefe_un = fe_unadj.std_errors
covfe_un= fe_unadj.cov

# RE (unadjusted): from your FGLS OLS-on-transformed
bre_un  = re_fgls.params
sere_un = re_fgls.bse
covre_un= re_fgls.cov_params()

t_table_un = hausman_t_per_coef(bfe_un, sefe_un, bre_un, sere_un, slope_names)
H_un = hausman_joint(bfe_un, covfe_un, bre_un, covre_un, slope_names, tag="unadjusted")

print("\n=== Hausman per-coefficient t-tests (UNADJUSTED, textbook 15.36) ===")
print(t_table_un.round(4))

print("\n=== Joint Hausman contrast (UNADJUSTED) ===")
print(f"H = {H_un['H']:.2f}  df = {H_un['df']}  p-value = {H_un['p']:.4g}  [{H_un['inv']}]")
print("diff vector d (b_FE - b_RE):", np.round(H_un["d"], 4))
# If you want to see V = Var_FE - Var_RE:
# print("V =\n", pd.DataFrame(H_un["V"], index=slope_names, columns=slope_names).round(6))

# 3) CLUSTER-ROBUST version (shown for completeness; the book warns it may not strictly apply)
# RE with clusters on individual id
groups = X_star.index.get_level_values(0)
re_clus = sm.OLS(y_star, X_star).fit(cov_type="cluster", cov_kwds={"groups": groups})
# FE with clustered SEs
fe_clus = PanelOLS(df[YCOL], X_fe, entity_effects=True)\
            .fit(cov_type="clustered", cluster_entity=True)

bfe_cl, covfe_cl = fe_clus.params, fe_clus.cov
bre_cl, covre_cl = re_clus.params, re_clus.cov_params()
# restrict to common slopes again (in case clustering drops anything)
common_cl = pd.Index(bfe_cl.index).intersection(bre_cl.index)
slope_names_cl = [c for c in common_cl if c.lower() != "const"]

H_cl, df_cl, p_cl, inv_kind_cl, d_cl, V_cl = hausman_joint(
    bfe_cl, covfe_cl, bre_cl, covre_cl, slope_names_cl, tag="clustered"
)

H_cl = hausman_joint(bfe_cl, covfe_cl, bre_cl, covre_cl, slope_names_cl, tag="clustered")

print("\n=== Joint Hausman contrast (CLUSTER-ROBUST; reference only) ===")
print(f"H = {H_cl['H']:.2f}  df = {H_cl['df']}  p-value = {H_cl['p']:.4g}  [{H_cl['inv']}]")
print("diff vector d (b_FE - b_RE):", np.round(H_cl["d"], 4))


Estimated components: sigma_u^2=0.607958, sigma_e^2=0.147907
Partial-demeaning coefficient alpha = 0.7261

=== RANDOM EFFECTS (Unadjusted SEs) ===
                            OLS Regression Results                            
Dep. Variable:                 lsales   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     746.7
Date:                Mon, 15 Sep 2025   Prob (F-statistic):          7.39e-264
Time:                        15:04:28   Log-Likelihood:                -1371.5
No. Observations:                3000   AIC:                             2749.
Df Residuals:                    2997   BIC:                             2767.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.

In [40]:
# === Random Effects (FGLS) for nls with FGLS SEs and Cluster-Robust SEs ===
import numpy as np, pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS # Import PanelOLS for Fixed Effects

# Assumes you already have a DataFrame `nls` with columns:
# 'firm', 'year', 'lwage', 'educ', 'exper', 'tenure', 'south', 'black', 'union'
df_nls = nls.copy()

# Create squared terms for experience and tenure
df_nls['exper_squared'] = df_nls['exper'] ** 2
df_nls['tenure_squared'] = df_nls['tenure'] ** 2

# Ensure balanced-ish panel metadata
ID, TIME = "id", "year"
YCOL, XCOLS = "lwage", ["educ", 'exper', 'exper_squared', 'tenure', 'tenure_squared', 'south', 'black', 'union']

# Sort and set (firm,year) index
df_nls = df_nls[[ID, TIME, YCOL] + XCOLS].dropna().sort_values([ID, TIME]).set_index([ID, TIME])

# Count periods per firm (T); Example 15.9 uses T=3; code handles unbalanced too
Ti = df_nls.groupby(level=0).size()
# We'll use each firm's own T for moments; if perfectly balanced, Ti.nunique()==1
# Add a constant (intercept) for pooled OLS and the transformed model
def add_const(M):
    return sm.add_constant(M, has_constant="add")

# ---------- 1) Pooled OLS residuals (for Swamy–Arora variance components) ----------
y = df_nls[YCOL]
X = add_const(df_nls[XCOLS])
ols = sm.OLS(y, X).fit()

ehat = ols.resid
# Firm means of residuals and counts
eibar = ehat.groupby(level=0).mean()
Ti = ehat.groupby(level=0).size()

# Within (idiosyncratic) variance: sum over (e_it - ebar_i)^2 / (sum_i (T_i-1) - Kx)
demeaned = ehat - ehat.groupby(level=0).transform("mean")
Kx = X.shape[1]                          # includes constant
den_within = (Ti - 1).sum() - (Kx - 1)   # subtract slopes, constant handled by between; mild dof choice
sigma2_e = (demeaned**2).sum() / den_within

# Between variance of firm means: max(0, Var(ē_i) - sigma2_e / T_i averaged)
# Use weighted average of 1/T_i for unbalanced panels
w = 1.0 / Ti
sigma2_u = ( (eibar**2).mean() - (sigma2_e * w.mean()) ).clip(min=0)

# ---------- 2) Compute alpha for partial demeaning ----------
# If panel is unbalanced, replace T by firm-specific T_i inside the transform.
# For a simple, Wooldridge-style single-alpha, use average T:
Tbar = Ti.mean()
alpha = 1.0 - np.sqrt(sigma2_e) / np.sqrt(Tbar * sigma2_u + sigma2_e)

# Guardrails
alpha = float(np.clip(alpha, 0.0, 1.0))

print(f"Estimated components: sigma_u^2={sigma2_u:.6f}, sigma_e^2={sigma2_e:.6f}")
print(f"Partial-demeaning coefficient alpha = {alpha:.4f}")

# ---------- 3) Build RE-transformed (quasi-demeaned) variables ----------
# y* = y - alpha * ybar_i,   x* = x - alpha * xbar_i,   const* = 1 - alpha
group = df_nls.groupby(level=0)
y_star = y - alpha * group[YCOL].transform("mean")

X_star = X.copy()
for col in X_star.columns:
    if col == "const":                  # mean of constant is 1, so const* = 1 - alpha
        X_star[col] = 1.0 - alpha
    else:
        X_star[col] = X_star[col] - alpha * group[col].transform("mean")

# ---------- 4) OLS on transformed data (== RE / FGLS) ----------
# (a) Model-based FGLS SEs (non-robust)
re_fgls = sm.OLS(y_star, X_star).fit()

# ---------- 5) Fixed Effects model using PanelOLS ----------
# Use the original y and X without the added constant for PanelOLS with EntityEffects
# Add a constant term explicitly for PanelOLS
X_fe = sm.add_constant(df_nls[XCOLS], has_constant='add')
fe_model = PanelOLS(df_nls[YCOL], X_fe, entity_effects=True, drop_absorbed=True).fit()

# ---------- 6) Print Summaries and Compact comparison table ----------
print("\n=== RANDOM EFFECTS (non-robust SEs) ===")
print(re_fgls.summary())

print("\n=== FIXED EFFECTS (non-robust SEs) ===")
print(fe_model.summary)

def tidy(res, tag):
    out = pd.DataFrame({"coef": res.params, "se": res.bse})
    out["t"] = out["coef"] / out["se"]
    out["spec"] = tag
    return out

# For the FE model, summarize the results
fe_params = fe_model.params
fe_std_errors = fe_model.std_errors
fe_tstats = fe_model.tstats

fe_tidy = pd.DataFrame({
    "coef": fe_params,
    "se": fe_std_errors,
    "t": fe_tstats
})
fe_tidy["spec"] = "fe_clus"

# Combine and compare Random Effects vs Fixed Effects
tbl = pd.concat([tidy(re_fgls, "re_fgls"), fe_tidy]).reset_index().rename(columns={"index":"param"})
print("\nRandom Effects vs Fixed Effects — coefficients and t-stats (Cluster SEs):\n")
print(tbl.pivot(index="param", columns="spec", values=["coef","se","t"]).round(4))

# =========================
# Hausman per-coef t-tests + joint contrast (UNADJUSTED = textbook)
# =========================
from scipy.stats import chi2
import numpy as np
import pandas as pd

# keep only coefficients present in both models; drop the intercept
common = pd.Index(re_fgls.params.index).intersection(fe_model.params.index)
slope_names = [c for c in common if c.lower() != "const"]

# helpers
def hausman_t_per_coef(beta_fe, se_fe, beta_re, se_re, labels):
    """ t_k = (b_FE - b_RE) / sqrt(se_FE^2 - se_RE^2) """
    rows = []
    for k in labels:
        num  = beta_fe[k] - beta_re[k]
        den2 = se_fe[k]**2 - se_re[k]**2
        tval = np.nan if den2 <= 0 else num / np.sqrt(den2)
        rows.append({"var": k, "diff(b_FE-b_RE)": num, "t (unadj)": tval,
                     "se_FE": se_fe[k], "se_RE": se_re[k]})
    return pd.DataFrame(rows).set_index("var")

def hausman_joint(beta_fe, cov_fe, beta_re, cov_re, labels):
    """ H = d' (Var_FE - Var_RE)^(-1) d  ~  chi2_df with df=len(labels) """
    bfe = beta_fe.loc[labels].to_numpy()
    bre = beta_re.loc[labels].to_numpy()
    d   = bfe - bre
    V   = cov_fe.loc[labels, labels].to_numpy() - cov_re.loc[labels, labels].to_numpy()
    try:
        Vinv = np.linalg.inv(V)
        inv_used = "standard inverse"
    except np.linalg.LinAlgError:
        Vinv = np.linalg.pinv(V)
        inv_used = "pseudo-inverse"
    H  = float(d.T @ Vinv @ d)
    df = len(labels)
    p  = 1.0 - chi2.cdf(H, df)
    return H, df, p, inv_used, d, V

# ----- UNADJUSTED (matches textbook derivation) -----
bfe_un   = fe_model.params
sefe_un  = fe_model.std_errors
covfe_un = fe_model.cov

bre_un   = re_fgls.params
sere_un  = re_fgls.bse
covre_un = re_fgls.cov_params()

t_un = hausman_t_per_coef(bfe_un, sefe_un, bre_un, sere_un, slope_names)
H_un, df_un, p_un, inv_kind_un, d_un, V_un = hausman_joint(bfe_un, covfe_un, bre_un, covre_un, slope_names)

print("\n=== Hausman per-coefficient t-tests (UNADJUSTED) ===")
print(t_un.round(4))

print("\n=== Joint Hausman contrast (UNADJUSTED) ===")
print(f"H = {H_un:.2f}   df = {df_un}   p-value = {p_un:.4g}   [{inv_kind_un}]")
print("diff vector d (b_FE - b_RE):", np.round(d_un, 4))



Estimated components: sigma_u^2=0.106828, sigma_e^2=0.038797
Partial-demeaning coefficient alpha = 0.7398

=== RANDOM EFFECTS (non-robust SEs) ===
                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.195
Model:                            OLS   Adj. R-squared:                  0.194
Method:                 Least Squares   F-statistic:                     108.3
Date:                Mon, 15 Sep 2025   Prob (F-statistic):          2.22e-162
Time:                        15:04:29   Log-Likelihood:                 756.88
No. Observations:                3580   AIC:                            -1496.
Df Residuals:                    3571   BIC:                            -1440.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|     

/tmp/ipython-input-3450000173.py:81: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

educ, black

  fe_model = PanelOLS(df_nls[YCOL], X_fe, entity_effects=True, drop_absorbed=True).fit()


# Mundlak Regressions for a Production Function

In [41]:
chemical3.columns

Index(['firm', 'year', 'lsales', 'lcapital', 'llabor', 'sk_labor',
       'lmaterials', 'foreign', 'export', 'intangibles', 'ownership'],
      dtype='object')

In [42]:
# ===== Mundlak (Correlated Random Effects) for chemical3 =====
import numpy as np, pandas as pd, statsmodels.api as sm
from linearmodels.panel import RandomEffects

# --- 0) Data prep: MultiIndex (firm, year) and core variables
df = chemical3.copy()
df = df[['firm','year','lsales','lcapital','llabor']].dropna().sort_values(['firm','year'])
df = df.set_index(['firm','year'])

y = df['lsales']
X_tv = df[['lcapital','llabor']]  # time-varying regressors

# --- 1) Firm means of time-varying regressors (the Mundlak "bars")
X_bar = X_tv.groupby(level=0).transform('mean').add_suffix('_bar')

# Design matrices
X_all = pd.concat([X_tv, X_bar], axis=1)
X_ols = sm.add_constant(X_all, has_constant='add')

# --- 2) Pooled OLS with cluster-robust SEs by firm (book says OLS works & SEs = RE cluster SEs)
firm_ids = X_ols.index.get_level_values(0)
ols_clu = sm.OLS(y, X_ols).fit(cov_type="cluster", cov_kwds={"groups": firm_ids})

# (Optional) conventional OLS (non-robust) if you want to compare
ols_conv = sm.OLS(y, X_ols).fit()

# --- 3) Random Effects on the same Mundlak regressors
# Conventional (unadjusted) SEs
re_conv = RandomEffects(y, X_ols).fit(cov_type="unadjusted")
# Cluster-robust (by entity) SEs
re_clu  = RandomEffects(y, X_ols).fit(cov_type="clustered", cluster_entity=True)

# --- 4) Build a Table 15.6 style summary
rows = ['const','lcapital','llabor','lcapital_bar','llabor_bar']

tab = pd.DataFrame({
    'OLS Coef': ols_clu.params.reindex(rows),
    'Cluster SE (OLS)': ols_clu.bse.reindex(rows),
    'RE Coef (conv)': re_conv.params.reindex(rows),
    'Conv. SE (RE)': re_conv.std_errors.reindex(rows),
    'RE Coef (cluster)': re_clu.params.reindex(rows),
    'Cluster SE (RE)': re_clu.std_errors.reindex(rows),
})

print("\n=== Mundlak regressions for chemical3 (replicating Example 15.14 / Table 15.6) ===")
print(tab.round(6))

# --- 5) Mundlak joint test: H0: all mean terms = 0  (γ2 = γ3 = 0)
# (a) Cluster-robust Wald using OLS (clustered) — matches the book's 'cluster-robust standard errors'
param_names = ols_clu.params.index.tolist()
mean_params = [p for p in param_names if p.endswith('_bar')]

R = np.zeros((len(mean_params), len(param_names)))
for i, p in enumerate(mean_params):
    R[i, param_names.index(p)] = 1.0

wald_clu = ols_clu.wald_test(R, scalar=True)
chi2_clu = float(wald_clu.statistic)

# degrees of freedom = number of restrictions (rows of R)
df_clu   = R.shape[0]

p_clu    = float(wald_clu.pvalue)



# (b) Conventional (unadjusted) Wald using RE (unadjusted)
# linearmodels returns cov in re_conv.cov; compute Wald by hand
beta = re_conv.params.reindex(rows).to_numpy()
cov  = re_conv.cov.loc[rows, rows].to_numpy()
# pick the rows/cols for the mean terms
pick = [rows.index(p) for p in ['lcapital_bar','llabor_bar']]
R_re = np.zeros((len(pick), len(rows)))
for i, j in enumerate(pick):
    R_re[i, j] = 1.0

Rb   = R_re @ beta
V_R  = R_re @ cov @ R_re.T
try:
    V_R_inv = np.linalg.inv(V_R)
except np.linalg.LinAlgError:
    V_R_inv = np.linalg.pinv(V_R)
chi2_conv = float(Rb.T @ V_R_inv @ Rb)
df_conv   = R_re.shape[0]

# --- 6) Print tests
print("\n=== Mundlak joint test: H0: lcapital_bar = llabor_bar = 0 ===")
print(f"Cluster-robust (OLS with cluster SEs):  Chi2({df_clu}) = {chi2_clu:.2f},  p = {p_clu:.4g}")
print(f"Conventional (RE unadjusted SEs):      Chi2({df_conv}) = {chi2_conv:.2f}")

# --- 7) Small sanity notes (what to expect)
print("\nNotes:")
print("• OLS and RE coefficients on all Mundlak terms should be identical (they estimate the same model).")
print("• The cluster-robust and conventional standard errors differ; the book reports both.")
print("• The ‘within’ effects (lcapital, llabor) should be close to your FE estimates;")
print("  significant mean terms ⇒ RE exogeneity fails ⇒ prefer FE (same conclusion as Hausman).")



=== Mundlak regressions for chemical3 (replicating Example 15.14 / Table 15.6) ===
              OLS Coef  Cluster SE (OLS)  RE Coef (conv)  Conv. SE (RE)  \
const         5.455328          0.148417        5.455328       0.137178   
lcapital      0.116040          0.027351        0.116040       0.019546   
llabor        0.268880          0.045825        0.268880       0.030668   
lcapital_bar  0.222320          0.041255        0.222320       0.033390   
llabor_bar    0.109495          0.062204        0.109495       0.050105   

              RE Coef (cluster)  Cluster SE (RE)  
const                  5.455328         0.148368  
lcapital               0.116040         0.027342  
llabor                 0.268880         0.045809  
lcapital_bar           0.222320         0.041241  
llabor_bar             0.109495         0.062184  

=== Mundlak joint test: H0: lcapital_bar = llabor_bar = 0 ===
Cluster-robust (OLS with cluster SEs):  Chi2(2) = 56.59,  p = 5.159e-13
Conventional (RE unadjus

# The Mundlak Approach for a Wage Equation

In [71]:
# ===== Mundlak for the NLS wage equation (like Wooldridge Example 15.15) =====
import numpy as np, pandas as pd, statsmodels.api as sm
from linearmodels.panel import RandomEffects, PanelOLS

# ---- 0) Data ----
df = nls_panel.copy() if 'nls_panel' in globals() else nls.copy()
keep = ['id','year','lwage','educ','exper','tenure','south','black','union']
df = df[keep].dropna().sort_values(['id','year']).copy()
# ensure numeric
for c in keep: df[c] = pd.to_numeric(df[c], errors='coerce')
df = df.dropna().set_index(['id','year'])

# time-varying pieces and their squares
df['exper2']  = df['exper']**2
df['tenure2'] = df['tenure']**2

# sets
Z_time_invar = ['educ','black']                            # in FE these are absorbed
X_tv         = ['exper','exper2','tenure','tenure2','south','union']  # vary over time

# ---- 1) Mundlak: firm means of time-varying regressors ----
X_bar = df[X_tv].groupby(level=0).transform('mean').add_suffix('_bar')

# ---- 2) Design matrices ----
# OLS/RE Mundlak specification: const + Z + X_tv + X_bar
X_all = pd.concat([df[Z_time_invar], df[X_tv], X_bar], axis=1)
X_ols = sm.add_constant(X_all, has_constant='add')
y     = df['lwage']

# ---- 3) OLS (cluster-robust by person) & Random Effects (conv + cluster) ----
pid = X_ols.index.get_level_values('id')

ols_clu = sm.OLS(y, X_ols).fit(cov_type="cluster", cov_kwds={"groups": pid})
ols_conv = sm.OLS(y, X_ols).fit()

re_conv = RandomEffects(y, X_ols).fit(cov_type="unadjusted")
re_clu  = RandomEffects(y, X_ols).fit(cov_type="clustered", cluster_entity=True)

# ---- 4) Fixed Effects for comparison (cluster SEs) ----
# FE on time-varying regressors only; time-invariant terms are absorbed
X_fe = sm.add_constant(df[X_tv], has_constant='add')
fe_clu = PanelOLS(y, X_fe, entity_effects=True).fit(cov_type="clustered", cluster_entity=True)

# ---- 5) Table like 15.7 ----
rows = ['const','educ','exper','exper2','tenure','tenure2','black','south','union',
        'exper_bar','tenure_bar','south_bar','union_bar']

tab = pd.DataFrame({
    'RE (conv) Coef'   : re_conv.params.reindex(rows),
    'RE (conv) SE'     : re_conv.std_errors.reindex(rows),
    'RE (cluster) Coef': re_clu.params.reindex(rows),
    'RE (cluster) SE'  : re_clu.std_errors.reindex(rows),
})

# FE coefficients exist only for time-varying originals
fe_cols = ['const','exper','exper2','tenure','tenure2','south','union']
fe_block = pd.DataFrame({
    'FE (cluster) Coef': fe_clu.params.reindex(fe_cols),
    'FE (cluster) SE'  : fe_clu.std_errors.reindex(fe_cols)
})
tab = tab.join(fe_block, how='left')

print("\n=== Mundlak regressions for NLS wage equation (Example 15.15 style) ===")
print(tab.round(4))

# ---- 6) Mundlak joint Wald tests: H0: all mean terms = 0 ----
# (a) Cluster-robust version using OLS cluster covariance
param_names = ols_clu.params.index.tolist()
mean_params = [p for p in param_names if p.endswith('_bar')]
R = np.zeros((len(mean_params), len(param_names)))
for i,p in enumerate(mean_params):
    R[i, param_names.index(p)] = 1.0

wald_clu = ols_clu.wald_test(R, scalar=True)    # robust to heterosk/within-id corr
chi2_clu = float(wald_clu.statistic)
df_clu   = R.shape[0]                            # number of restrictions
p_clu    = float(wald_clu.pvalue)

# (b) Conventional (unadjusted) Wald using RE covariance
beta = re_conv.params.loc[param_names].to_numpy()
cov  = re_conv.cov.loc[param_names, param_names].to_numpy()
Rb   = R @ beta
V_R  = R @ cov @ R.T
V_R_inv = np.linalg.pinv(V_R)                    # safe inverse
chi2_conv = float(Rb.T @ V_R_inv @ Rb)
df_conv   = R.shape[0]

print("\n=== Mundlak joint test: H0: exper_bar = tenure_bar = south_bar = union_bar = 0 ===")
print(f"Cluster-robust (OLS):   Chi2({df_clu}) = {chi2_clu:.2f},  p = {p_clu:.4g}")
print(f"Conventional (RE):      Chi2({df_conv}) = {chi2_conv:.2f}")

# ---- 7) Quick interpretive notes ----
print("\nNotes:")
print("• RE and OLS coefficients in the Mundlak model should match; only SEs differ (cluster vs conventional).")
print("• FE matches RE/OLS for time-varying variables; EDUC and BLACK are time-invariant, so FE omits them.")
print("• If the Mundlak test is significant, unobserved effects correlate with regressors → prefer FE for causal interpretation.")



=== Mundlak regressions for NLS wage equation (Example 15.15 style) ===
            RE (conv) Coef  RE (conv) SE  RE (cluster) Coef  RE (cluster) SE  \
const               0.4167        0.1361             0.4167           0.1101   
educ                0.0708        0.0054             0.0708           0.0056   
exper               0.0411        0.0066             0.0411           0.0082   
exper2             -0.0004        0.0003            -0.0004           0.0003   
tenure              0.0139        0.0033             0.0139           0.0042   
tenure2            -0.0009        0.0002            -0.0009           0.0002   
black              -0.1216        0.0317            -0.1216           0.0284   
south              -0.0163        0.0361            -0.0163           0.0585   
union               0.0637        0.0142             0.0637           0.0169   
exper_bar           0.0251        0.0244             0.0251           0.0223   
tenure_bar          0.0026        0.0127       

In [74]:
nls.columns

Index(['id', 'year', 'lwage', 'hours', 'age', 'educ', 'collgrad', 'msp',
       'nev_mar', 'not_smsa', 'c_city', 'south', 'black', 'union', 'exper',
       'exper2', 'tenure', 'tenure2'],
      dtype='object')

2
